In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.sparse import csr_matrix,hstack
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from collections import defaultdict

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import sys

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/iterative-stratification/iterative-stratification-master/.gitignore
/kaggle/input/iterative-stratification/iterative-stratification-master/LICENSE
/kaggle/input/iterative-stratification/iterative-stratification-master/README.md
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.cfg
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.py
/kaggle/input/iterative-stratification/iterative-stratification-master/.travis.yml
/kaggle/input/iterative-stratification/iterative-stratification-master/tests/__init__.py
/kaggle/input/iterative-stratification/iterative-stratification-master/tests/test_ml_stratifiers.py
/kaggle/input/iterative-stratification/iterative-stratification-master/iterstrat/__init__.py
/kagg

In [2]:
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [3]:
train_features=pd.read_csv('../input/lish-moa/train_features.csv')
test_features=pd.read_csv('../input/lish-moa/test_features.csv')
train_targets_scored=pd.read_csv('../input/lish-moa/train_targets_scored.csv')
sample_submission=pd.read_csv('../input/lish-moa/sample_submission.csv')

In [4]:
train_features.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [5]:
test_features.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


In [6]:
ctrl_vehicle_idx=test_features[test_features['cp_type']=='ctl_vehicle']['sig_id'].values
len(ctrl_vehicle_idx)

358

In [7]:
train_targets_scored.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
targets=train_targets_scored.columns[1:]
len(targets)

206

In [9]:
#kfolds=MultilabelStratifiedKFold(n_splits=5,random_state=42,shuffle=True)
kfolds=StratifiedKFold(n_splits=3,random_state=42,shuffle=True)

In [10]:
tar=['5-alpha_reductase_inhibitor','atp-sensitive_potassium_channel_antagonist','11-beta-hsd1_inhibitor']

### Build a classifier chain

In [11]:
#Reference https://www.kaggle.com/rhodiumbeng/classifying-multi-label-comments-0-9741-lb
def add_features(X_feat,feat_to_add):
    """
    Return sparse matrix with added features.
    """
    return hstack([X_feat.astype(float),csr_matrix(feat_to_add).T],'csr')

In [12]:
def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip)))
    return loss

In [13]:
imp_feats=['c-30',
'c-34',
'c-46',
'c-6',
'c-65',
'c-67',
'c-70',
'c-76',
'c-78',
'c-9',
'c-98',
'cp_type_ctl_vehicle',
'g-100',
'g-108',
'g-117',
'g-122',
'g-138',
'g-148',
'g-156',
'g-157',
'g-158',
'g-167',
'g-168',
'g-170',
'g-175',
'g-178',
'g-181',
'g-183',
'g-20',
'g-202',
'g-203',
'g-206',
'g-207',
'g-208',
'g-217',
'g-22',
'g-231',
'g-235',
'g-269',
'g-291',
'g-30',
'g-300',
'g-332',
'g-34',
'g-357',
'g-377',
'g-385',
'g-387',
'g-392',
'g-414',
'g-418',
'g-431',
'g-441',
'g-443',
'g-48',
'g-489',
'g-493',
'g-512',
'g-522',
'g-528',
'g-557',
'g-558',
'g-57',
'g-570',
'g-58',
'g-62',
'g-628',
'g-634',
'g-635',
'g-639',
'g-640',
'g-656',
'g-670',
'g-708',
'g-75',
'g-759',
'g-761',
'g-763',
'g-8',
'g-84',
'g-91']
len(imp_feats)

81

In [14]:
train_df=pd.concat([train_features,pd.get_dummies(train_features['cp_time'],prefix='cp_time')],axis=1)
train_df=pd.concat([train_df,pd.get_dummies(train_df['cp_type'],prefix='cp_type')],axis=1)
train_df=pd.concat([train_df,pd.get_dummies(train_df['cp_dose'],prefix='cp_dose')],axis=1)
train_df=train_df.drop(['cp_time','cp_type','cp_dose'],axis=1)


test_df=pd.concat([test_features,pd.get_dummies(test_features['cp_time'],prefix='cp_time')],axis=1)
test_df=pd.concat([test_df,pd.get_dummies(test_df['cp_type'],prefix='cp_type')],axis=1)
test_df=pd.concat([test_df,pd.get_dummies(test_df['cp_dose'],prefix='cp_dose')],axis=1)
test_df=test_df.drop(['cp_time','cp_type','cp_dose'],axis=1)

test_df=test_df.loc[:,imp_feats]
train_df=train_df.loc[:,imp_feats]


In [15]:
score_dict=defaultdict(int)
scores=[]
zero_preds=[]    
class_preds=np.zeros(train_targets_scored.shape[0])
scaled=StandardScaler().fit(train_df)
train_df_scaled=scaled.transform(train_df)
test_df_scaled=scaled.transform(test_df)
for t_ in targets:
    train_targets=train_targets_scored[t_]
    cv_scores=[]
    if train_targets.sum()>=5:
        for fold_,(trn_,val_) in enumerate(kfolds.split(train_df_scaled,train_targets)):
            print(f'-------Started training fold {fold_}---------')
            #print(train_df.iloc[trn_])
            #print(train_targets.iloc[trn_])
            trn_x,trn_y=train_df.iloc[trn_].values,train_targets.iloc[trn_].values
            val_x,val_y=train_df.iloc[val_].values,train_targets.iloc[val_].values
            classifier=LogisticRegression(C=5,solver='sag',n_jobs=-1,class_weight='balanced')
            classifier.fit(trn_x,trn_y)
            class_preds[val_]=classifier.predict_proba(val_x)[:,1]
            score=log_loss_metric(train_targets[val_],class_preds[val_])
            print(f'Fold {fold_} Class name {t_} log loss Score {score}')
            classifier.fit(train_df,train_targets)
            sample_submission[t_]+=classifier.predict_proba(test_df_scaled)[:,1]/5
            cv_scores.append(score)
            
    else:
        print(f'Class {t_} is not predicted')
        zero_preds.append(t_)
    #     print(f'Joining {t_} in the train dataframe')
    #     train_df=add_features(train_df,train_targets)
    #     print(train_df.shape)
    #     print(f'Joining {t_} in the test dataframe')
    #     test_df=add_features(test_df,sample_submission[t_])
    #     print(test_df.shape)

    print(f'----For class {t_} Overall Score {np.mean(cv_scores)}------')

        # try:
    scores.append(log_loss_metric(train_targets,class_preds))
    score_dict[t_]=log_loss_metric(train_targets,class_preds)
        # except ValueError:
        #     scores.append(0)
        #     score_dict[class_name]=0
        # print(f'Class {class_name} Full AUC Score  {roc_auc_score(train_targets,class_preds)}')
        # train_targets_scored[class_name+'_oof']=class_preds

print(f'Total CV Score {np.mean(cv_scores)}')

-------Started training fold 0---------
Fold 0 Class name 5-alpha_reductase_inhibitor log loss Score 0.06404696192018235
-------Started training fold 1---------
Fold 1 Class name 5-alpha_reductase_inhibitor log loss Score 0.0669996377125985
-------Started training fold 2---------
Fold 2 Class name 5-alpha_reductase_inhibitor log loss Score 0.09749134835979353
----For class 5-alpha_reductase_inhibitor Overall Score 0.0761793159975248------
-------Started training fold 0---------
Fold 0 Class name 11-beta-hsd1_inhibitor log loss Score 0.10551016382979209
-------Started training fold 1---------
Fold 1 Class name 11-beta-hsd1_inhibitor log loss Score 0.11972717570622667
-------Started training fold 2---------
Fold 2 Class name 11-beta-hsd1_inhibitor log loss Score 0.08853476112920149
----For class 11-beta-hsd1_inhibitor Overall Score 0.1045907002217401------
-------Started training fold 0---------
Fold 0 Class name acat_inhibitor log loss Score 0.6658142190169017
-------Started training fo

In [16]:
score_dict

defaultdict(int,
            {'5-alpha_reductase_inhibitor': 0.07617931599752416,
             '11-beta-hsd1_inhibitor': 0.10459070022174043,
             'acat_inhibitor': 0.501128984096009,
             'acetylcholine_receptor_agonist': 0.5538888214266283,
             'acetylcholine_receptor_antagonist': 0.5942062652589937,
             'acetylcholinesterase_inhibitor': 0.658185186674427,
             'adenosine_receptor_agonist': 0.5149002773654973,
             'adenosine_receptor_antagonist': 0.5787976519723281,
             'adenylyl_cyclase_activator': 0.007946201552619828,
             'adrenergic_receptor_agonist': 0.5395590978763692,
             'adrenergic_receptor_antagonist': 0.6008881884959851,
             'akt_inhibitor': 1.6532988362254473,
             'aldehyde_dehydrogenase_inhibitor': 0.07327757128347766,
             'alk_inhibitor': 1.0831606883560234,
             'ampk_activator': 0.053943631491578034,
             'analgesic': 0.056905035141807156,
         

In [17]:
zero_preds

['atp-sensitive_potassium_channel_antagonist', 'erbb2_inhibitor']

In [18]:
sample_submission.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.500155,0.500014,0.916548,1.092698,0.989314,0.961028,0.977184,1.055565,0.500002,...,1.091263,0.500228,0.942559,0.513271,0.500627,0.500000,0.507910,1.089591,0.548860,0.511299
1,id_001897cda,0.500000,0.500000,0.500101,0.841197,0.861981,0.959833,0.603969,1.075709,0.500839,...,0.500005,0.500000,0.922349,0.510771,0.992704,0.500005,1.047532,1.066967,0.500004,0.539997
2,id_002429b5b,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.500015,0.500000,0.500000,0.500004,0.500000,0.500000,0.500000
3,id_00276f245,0.500007,0.500001,0.500052,1.011041,1.021033,0.983756,1.016883,1.000663,0.500118,...,0.500000,0.500000,1.043266,0.646186,0.559350,0.500081,0.502398,0.515849,0.500001,0.500747
4,id_0027f1083,1.042575,0.506039,0.690800,1.055854,1.041740,0.994161,1.095959,0.914299,0.500001,...,0.500010,0.701931,1.085824,0.552778,0.796490,0.500032,0.511116,0.503179,0.501342,0.704208


### Post Processing

In [19]:
sample_submission.loc[:,zero_preds]=0
sample_submission.loc[sample_submission['sig_id'].isin(ctrl_vehicle_idx),1:]=0

In [20]:
sample_submission.to_csv('submission.csv',index=False)